In [ ]:
# default_exp analyze_by_person

# Analysis By Person

> Module processes actiwatch data for a single individual, with the ability to add additional astral (sunset/sunrise) and sleep timing data.

In [ ]:
#hide
from nbdev.showdoc import *

%run load_actiwatch_data.py
%run firsttime.py

import numpy as np
import pandas as pd

from joblib import *
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

## Loading Actiwatch Data

Actiwatch data is best served loaded in a directory style setup with key value pairings. This is intended to provide a generally flexible method for labeling groups within the data for easier by-group searching and analysis.

For example:

The below directory uses

- **key** = indicates the group name (for example, base_ or follow_up_), that is used to both name the grouping within the Group column, and as the front of the generated UID.
       
       
 - **value** = indicates that the csv files to be loaded are stored in a folder called "data/v1 or data/v3". The trailing part of the naming convention (after the / ) is appended to the UID. The remaining part of the UID is built using the file name within the subfolder.
        
    Following this structure an example file titled 'user1234' would net a UID of *base_v1\user1234* as
    its key is *base_*, its stored in *v1*, and the title of the csv is *user1234*.
    
    Also note that the name for their "group" will match the key value used for the directory.

In [ ]:
directory = {
    'base_': 'data/v1',
    'follow_up_': 'data/v3'
}

In [ ]:
#export
def get_raw_data(key:str, directory: dict, grouping:str = 'Group'):
    """Loads raw actiwatch data for a particular group based on a string key.

    #### Parameters

    key: str

        The key to load actiwatch data from (for example, "v1")
    directory: dict

        Dictionary of valid folders to load actiwatch data from.
        Folders should have .csv files in them.
    grouping: str

        Name of the generated column for specifying groupings, where
        the values will be the name of the key given. Default = 'Group'

    """
    raw_data, summary_data = load_actiwatch_data(directory[key],uidprefix = key)
    raw_data[grouping] = key
    return raw_data

In [ ]:
show_doc(get_raw_data, title_level = 3)

<h3 id="get_raw_data" class="doc_header"><code>get_raw_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>get_raw_data</code>(**`key`**:`str`, **`directory`**:`dict`, **`grouping`**:`str`=*`'Group'`*)

Loads raw actiwatch data for a particular group based on a string key.

#### Parameters

key: str

    The key to load actiwatch data from (for example, "v1")
directory: dict

    Dictionary of valid folders to load actiwatch data from.
    Folders should have .csv files in them.
grouping: str

    Name of the generated column for specifying groupings, where
    the values will be the name of the key given. Default = 'Group'

#### Example

An example of output for this function would be:

In [ ]:
raw_data = get_raw_data('base_', directory)
raw_data.dropna().head() 

Found 1 csv files in data/v1/. Pass #1, raw data
.
.
Pass #2, data summary
.
.EOF without retrieving summary data: data/v1\user1234_v1sample.csv


,Off-Wrist Status,Activity,Marker,White Light,Red Light,Green Light,Blue Light,Sleep/Wake,Interval Status,UID,Group
DateTime,,,,,,,,,,,
2018-06-25 12:31:00,0,0.0,0.0,37.09,28.4,15.5,14.5,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:31:30,0,170.0,0.0,156.15,159.0,59.9,65.3,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:32:00,0,194.0,0.0,149.03,113.0,49.8,50.6,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:32:30,0,0.0,0.0,473.95,365.0,161.0,161.0,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:33:00,0,62.0,0.0,317.82,264.0,112.0,115.0,1.0,ACTIVE,base_v1\user1234,base_


## Exporting Timing Data

In [ ]:
#export
def export_timing_data(timing_data, outfile):
    """ Exports timing data to parquet.

    #### Parameters

    timing_data: pd.DataFrame

        Timing data
    """
    timing_data.Date = timing_data.Date.values.astype("datetime64[s]")
    timing_data.to_parquet(
        outfile + "timing.parquet", engine = "fastparquet", compression = "gzip"
       )

In [ ]:
show_doc(export_timing_data, title_level = 3)

<h3 id="export_timing_data" class="doc_header"><code>export_timing_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>export_timing_data</code>(**`timing_data`**)

Exports timing data to parquet.

#### Parameters

timing_data: pd.DataFrame

    Timing data

## Recalculating Timing Data (if Necessary)

Sometimes it may be valuable to calculate (or recalculate) the timing data. The ability to do so is provided using the below function, which sets up the timing and raw dataframes. 

In this example, we'll go ahead and use the city of Seattle as our location for determining sunlight (sunset and sunrise timings), and ask the function to recalculate both the raw and timing data. Example thresholds for lux are also specified below. The outfile specified is where the data will be written to.

In [ ]:
recalculate_raw = False 
recalculate_timing = False
thresholds = [ [5], [10], [50], [100], [500], [1000] ] 
outfile = "../SALA/example_output/"

In [ ]:
#export
def process_timing_data(
                     outfile: str,
                     thresholds: list,
                     directory: dict,
                     recalc_raw: bool = False,
                     recalc_timing: bool = False,
                     export_hook = None
                     ):
    """Setup timing and raw dataframes or recalculate their
    values if specified and necessary. Both operations take
    a long time and lots of memory. Ill-advised to recalculate
    timing specifically if the data has already been created.

    #### Parameters

    outfile: str

        File for re-written data to be placed in, or for data to be loaded from
    thresholds: list

        List of light thresholds for the watch data
    key: str

        The key to load actiwatch data from
    directory: dict

        Dictionary of valid seasons to retrieve actiwatch data from
    recalc_raw: bool

        Forces recalculation process if true, loads processed data from disk otherwise.
        Default value is 'False'
    recalc_timing: bool

        Forces recalculation of light timing data, loads it from disk otherwise.
        Default value is 'False'
    export_hook: function

        Placeholder for user to use their own function during data processing.
        This function should take in the timing data as a parameter. See
        documentation for example.
    #### Returns

        (as a tuple of pd.DataFrames) all the data, the timing data for a particular location
    """
    if recalc_raw:
        print("Loading raw data from disk...")
        raw_results = (
            Parallel(n_jobs=len(directory))(delayed(get_raw_data)(key, directory) for key in directory.keys())
                   )
        all_data = pd.concat(raw_results)
        # save data to parquet file
        all_data.to_parquet(outfile + "raw.parquet", engine = 'fastparquet',
                           compression = "gzip")
    else:
        # read data from parquet file
        all_data = pd.read_parquet(outfile + "raw.parquet")

    if recalc_timing:
        print("Calculating light timing data...")

        timing_results = (Parallel(n_jobs=len(thresholds))
            (delayed(firstAndLastLight)(all_data, threshold) for threshold in thresholds)
                      )
        timing_data = pd.concat(timing_results)
        print("Adding holiday markers to timing data...")
        cal = calendar()

        holidays = (
            cal.holidays(start = timing_data.Date.min(), end = timing_data.Date.max())
        )

        nn = pd.DatetimeIndex( timing_data.Date )
        timing_data["DayofWeek"] = nn.dayofweek
        days = ["Mon", "Tues", "Wed", "Thu", "Fri", "Sat", "Sun"]
        day_type = ["Weekday","Weekday","Weekday",
                    "Weekday","Weekday","Weekend/Holiday","Weekend/Holiday"]

        day_group = []
        dtp_group = []
        wknd_holiday = []

        # add days of week to data
        for index, row in timing_data.iterrows():
            day_group.append(row["Group"] + days[row["DayofWeek"]])
            if holidays.isin([row["Date"]]).any():
                dtp_group.append(row['Group'] + "Weekend/Holiday")
                wknd_holiday.append(True)
            else:
                dtp_group.append(
                    row["Group"] + day_type[row["DayofWeek"]]
                )
                wknd_holiday.append(row['DayofWeek'] > 4)

        timing_data["GroupDayofWeek"] = day_group
        timing_data["GroupDayType"] = dtp_group
        timing_data["Weekend/Holiday"] = wknd_holiday

        # function hook for extra processing before exporting to parquet
        if export_hook:
            timing_data = export_hook(timing_data)

        timing_copy = timing_data.copy()
        timing_copy["Watch period"] = pd.to_timedelta(timing_copy["Watch period"])
        export_timing_data(timing_copy, outfile)
    else:
        timing_copy = pd.read_parquet(outfile + "timing.parquet", engine = "fastparquet")
        # return date to original format
        timing_copy.Date = timing_copy.Date.apply(lambda x: x.date())
        timing_data = timing_copy.copy()

    return all_data, timing_data

In [ ]:
show_doc(process_timing_data, title_level = 3)

<h3 id="process_timing_data" class="doc_header"><code>process_timing_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>process_timing_data</code>(**`outfile`**:`str`, **`thresholds`**:`list`, **`directory`**:`dict`, **`recalc_raw`**:`bool`=*`False`*, **`recalc_timing`**:`bool`=*`False`*, **`export_hook`**=*`None`*)

Setup timing and raw dataframes or recalculate their
values if specified and necessary. Both operations take
a long time and lots of memory. Ill-advised to recalculate
timing specifically if the data has already been created.

#### Parameters

outfile: str

    File for re-written data to be placed in, or for data to be loaded from
thresholds: list

    List of light thresholds for the watch data
key: str

    The key to load actiwatch data from
directory: dict

    Dictionary of valid seasons to retrieve actiwatch data from
recalc_raw: bool

    Forces recalculation process if true, loads processed data from disk otherwise.
    Default value is 'False'
recalc_timing: bool

    Forces recalculation of light timing data, loads it from disk otherwise.
    Default value is 'False'
export_hook: function

    Placeholder for user to use their own function during data processing.
    This function should take in the timing data as a parameter. See
    documentation for example.
#### Returns

    (as a tuple of pd.DataFrames) all the data, the timing data for a particular location

### Function Hook Example

Before exporting timing data within the processing cycle, users can run their own custom analysis function to subset the data as desired. A very minimal example of such a function is provided below.

In [ ]:
#exports
def remove_first_day(timing_data):
    """Example function hook for removing data for the first day
    where its obvious that light data is non-existent (NaT)

     #### Parameters

    timing_data: pd.DataFrame

        Timing data
    """
    data = (
    timing_data[(timing_data["Last Light"].apply(np.isnat) == False)
               & (timing_data["Date"] != timing_data["Date"].min())]
            )
    return data

#### Example

An example of output for this function would be:

**Note**: timing_data is split into three images for easier viewing

In [ ]:
all_data, timing_data = process_timing_data(outfile, thresholds, directory, True, True, remove_first_day)

Loading raw data from disk...
Calculating light timing data...
Adding holiday markers to timing data...


In [ ]:
all_data.dropna().head()

,Off-Wrist Status,Activity,Marker,White Light,Red Light,Green Light,Blue Light,Sleep/Wake,Interval Status,UID,Group
DateTime,,,,,,,,,,,
2018-06-25 12:31:00,0,0.0,0.0,37.09,28.4,15.5,14.5,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:31:30,0,170.0,0.0,156.15,159.0,59.9,65.3,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:32:00,0,194.0,0.0,149.03,113.0,49.8,50.6,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:32:30,0,0.0,0.0,473.95,365.0,161.0,161.0,1.0,ACTIVE,base_v1\user1234,base_
2018-06-25 12:33:00,0,62.0,0.0,317.82,264.0,112.0,115.0,1.0,ACTIVE,base_v1\user1234,base_


In [ ]:
timing_data.iloc[:,:7].head()

,UID,Date,Threshold,Last Light,Mins to LL from 4AM,First Light,Mins to FL from 4AM
0,base_v1\user1234,2018-07-09,5,2018-07-09 23:19:00,1159.0,2018-07-09 06:39:00,159.0
1,base_v1\user1234,2018-07-07,5,2018-07-08 00:01:30,1201.0,2018-07-07 06:54:30,174.0
2,base_v1\user1234,2018-07-10,5,2018-07-10 08:08:00,248.0,2018-07-10 06:59:00,179.0
4,base_v1\user1234,2018-06-29,5,2018-06-29 22:18:30,1098.0,2018-06-29 06:47:00,167.0
5,base_v1\user1234,2018-07-06,5,2018-07-06 23:24:00,1164.0,2018-07-06 06:44:30,164.0


In [ ]:
timing_data.iloc[:,7:14].head()

,Time above threshold,Time above threshold AM,Minutes above threshold,Minutes above threshold AM,Lux minutes,Lux minutes AM,Group
0,0 days 13:56:00,0 days 04:35:30,836.0,275.5,1132776.310,678370.985,base_
1,0 days 11:45:30,0 days 04:37:30,705.5,277.5,814059.620,355668.225,base_
2,0 days 01:00:30,0 days 01:00:30,60.5,60.5,47044.555,47044.555,base_
4,0 days 05:00:30,0 days 03:26:30,300.5,206.5,65985.730,64188.250,base_
5,0 days 09:38:30,0 days 05:00:00,578.5,300.0,432815.220,298955.700,base_


In [ ]:
timing_data.iloc[:,14:].head()

,Watch period,DayofWeek,GroupDayofWeek,GroupDayType,Weekend/Holiday
0,0 days 00:00:30,0,base_Mon,base_Weekday,False
1,0 days 00:00:30,5,base_Sat,base_Weekend/Holiday,True
2,0 days 00:00:30,1,base_Tues,base_Weekday,False
4,0 days 00:00:30,4,base_Fri,base_Weekday,False
5,0 days 00:00:30,4,base_Fri,base_Weekday,False


## Setting Sunset and Sunrise Timings

Sunset and sunrise timings can also be calculated for actiwatch data. To do so, the specific location (longidtude and latitude) is required to be fed into the astral package. A label for the location (e.g. a city name like Seattle) is also helpful for clarity.

In [ ]:
#hide
from astral import LocationInfo, sun

In [ ]:
#export
def set_sun_timings(timing_data,
                    loc:str,
                    region: str,
                    timezone: str,
                    latitude: float,
                    longitude: float
                   ):
    """Given a location (city), calculate sunset and sunrise timings for the data

    #### Parameters

    timing_data: pd.DataFrame

        Timing data
    loc: str (any string)

        Name of location to lookup for sunrise/sunset calculations
    region: str (any string)

        Name of the region the location belongs to
    timezone: str

        the location's timezone (a list of timezones can be obtained from pytz.all_timezones)
    latitude: float

        Latitude position of the location for sunrise/sunset calculations
    longitude: float

        Longitude position of the location for sunrise/sunset calculations
    #### Returns

        Modified timing data with sunrise and sunset calculations

    """
    # add location info for calculating astral data
    city = LocationInfo(loc, region, timezone, latitude, longitude)

    timing_data["Sunrise"] = timing_data.Date.apply( lambda x: sun.sunrise(city.observer,
                                                                           x,
                                                                           tzinfo = city.tzinfo))

    timing_data["Sunset"] = timing_data.Date.apply( lambda x: sun.sunset(city.observer,
                                                                         x,
                                                                         tzinfo = city.tzinfo))


    return timing_data

In [ ]:
show_doc(set_sun_timings, title_level = 3)

<h3 id="set_sun_timings" class="doc_header"><code>set_sun_timings</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>set_sun_timings</code>(**`timing_data`**, **`loc`**:`str`, **`region`**:`str`, **`timezone`**:`str`, **`latitude`**:`float`, **`longitude`**:`float`)

Given a location (city), calculate sunset and sunrise timings for the data

#### Parameters

timing_data: pd.DataFrame

    Timing data
loc: str (any string)

    Name of location to lookup for sunrise/sunset calculations
region: str (any string)

    Name of the region the location belongs to
timezone: str

    the location's timezone (a list of timezones can be obtained from pytz.all_timezones)
latitude: float

    Latitude position of the location for sunrise/sunset calculations
longitude: float

    Longitude position of the location for sunrise/sunset calculations
#### Returns

    Modified timing data with sunrise and sunset calculations

#### Example

An example of the added portion to timing data from this function would be:

In [ ]:
timing_data = (
    set_sun_timings(timing_data, "Seattle", "United States", "America/Los_Angeles", 47.65, -122.30)
)

timing_data[["Sunrise", "Sunset"]].head()

,Sunrise,Sunset
0,2018-07-09 05:21:39.214048-07:00,2018-07-09 21:06:51.768689-07:00
1,2018-07-07 05:19:59.495724-07:00,2018-07-07 21:07:57.880755-07:00
2,2018-07-10 05:22:31.640667-07:00,2018-07-10 21:06:14.940775-07:00
4,2018-06-29 05:14:36.884762-07:00,2018-06-29 21:10:39.206298-07:00
5,2018-07-06 05:19:12.313737-07:00,2018-07-06 21:08:27.123116-07:00


## Adding Sleep Information

Adding sleep information to the timing data is also possible. The below function adds sleep data, allowing a "sleep day" to be split at a customizable time. The outputs of the function are
    
    1. short_frame: 
        a separate dataframe meant to be a quick way of visually subsetting and               viewing bi/polyphasic instances. This frame defaults to storing occurances 
        of at least 3 sleep periods within a "sleep day", but can be modified.
        
    2. timing_data:
        modifies input timing data to have all sleep period information
       

In [ ]:
#export
def process_sleep_data(timing_data, sleep_split: str = "18:00", num_sleeps: int = 3):
    """Processes sleep data for existing timing data.

    #### Parameters

    timing_data: pd.DataFrame

        Timing data
    sleep_split: str

        Time to split the sleep day. Default is "18:00", which is 6:00PM.
    num_sleeps: int

        Cutoff for number of sleeps to display in first resulting frame.
        Default = 3, frame will store days with 3+ sleep instances

    #### Returns

        short_frame: pd.DataFrame

            Onset, offset, and duration for sleep periods on days with
            more than num_sleeps number of sleep periods
        timing_data: pd.DataFrame

            Modified timing data with included sleep information

    """
    sleepers = []
    sleep_onsets = []
    sleep_offsets = []
    sleep_durations = []
    sleep_onsetMSLMs = []
    sleep_offsetMSLMs = []
    for arow in timing_data.itertuples():
        UID = arow.UID
        DT = pd.to_datetime(arow.Date)
        TM = pd.to_datetime(DT + pd.Timedelta("1 day"))
        today = DT.strftime("%Y-%m-%d")

        nextday = TM.strftime("%Y-%m-%d")

        # taking raw timing data entry and splitting a "sleep day" at 6pm
        # under the assumption that people do not end their days that early
        day_split = all_data.query("UID == @UID").loc[today +" " + sleep_split:nextday + " 18:00"]

        # REST-S = watch thinks user is asleep
        asleep = day_split[ day_split["Interval Status"] == "REST-S"].copy()

        # there may be more than one sleep period in a given day's data
        # new sleep period = when there is more than 1 hour between successive REST-S entries
        sleep_periods = []
        per = 0
        count = 0

        try:
            lt = asleep.index[0]
            for time in asleep.index:
                # allow up to 1 hour of being awake in the middle of the night
                if (time - lt > pd.Timedelta("1 hour")):
                    per += 1
                lt = time
                sleep_periods.append(per)
            asleep["Sleep period"] = sleep_periods
        except IndexError:
            asleep["Sleep period"] = [pd.to_datetime(0)]


        try:
        # calc sleep onsets/offsets/duration for each period of sleep in a person-day of data
            sleeps = asleep.reset_index().groupby("Sleep period").apply( lambda x: pd.DataFrame({
                     "Sleep onset": [x.DateTime.min()],
                     "Sleep offset": [x.DateTime.max()],
                     "Sleep duration": [x.DateTime.max() - x.DateTime.min()]
                     }, index = x.DateTime.dt.normalize() ))
        # if the value is = 0 -> np.int64 (not a DateTime)
        except AttributeError:
            sleeps = asleep.reset_index().groupby("Sleep period").apply( lambda x: pd.DataFrame({
             "Sleep onset": [pd.to_datetime(DT)],
             "Sleep offset": [pd.to_datetime(DT)],
             "Sleep duration": [pd.to_timedelta(x.DateTime.max() - x.DateTime.min())]
             }))
        sleeps = sleeps.drop_duplicates().sort_values(by="Sleep duration", ascending = False)
        onset = sleeps.iloc[0]['Sleep onset']
        offset = sleeps.iloc[0]['Sleep offset']
        dur =  sleeps.iloc[0]['Sleep duration']

        # if onset is actually a datetime
        if not isinstance(onset, np.int64):
            onMSLM = (onset - DT).total_seconds() / 60.0

        # if offset is actually a datetime
        if not isinstance(offset, np.int64):
            offMSLM = np.maximum((offset - TM).total_seconds() / 60.0, 0.0)

        sleep_onsets.append(onset)
        sleep_offsets.append(offset)
        sleep_durations.append(dur)
        sleep_onsetMSLMs.append(onMSLM)
        sleep_offsetMSLMs.append(offMSLM)
        sleep_count = sleeps.shape[0]

        # adding to short_frame
        if sleep_count >= num_sleeps:
            sleeps['UID'] = UID
            sleeps['DT'] = DT
            sleeps.reset_index(drop = True).set_index(['UID','DT'])
            sleepers.append(sleeps)
    short_frame = (
                   pd.concat(sleepers).reset_index().drop('DateTime',axis=1)
                   .set_index(['UID','DT']).drop_duplicates()
                   )
    timing_data["Sleep onset"] = sleep_onsets
    timing_data["Sleep offset"] = sleep_offsets
    timing_data["Sleep duration"] = sleep_durations
    timing_data["Sleep onset MSLM"] = sleep_onsetMSLMs
    timing_data["Sleep offset MSLM"] = sleep_offsetMSLMs

    return short_frame, timing_data

In [ ]:
show_doc(process_sleep_data, title_level = 3)

<h3 id="process_sleep_data" class="doc_header"><code>process_sleep_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>process_sleep_data</code>(**`timing_data`**, **`sleep_split`**:`str`=*`'18:00'`*, **`num_sleeps`**:`int`=*`3`*)

Processes sleep data for existing timing data.

#### Parameters

timing_data: pd.DataFrame

    Timing data
sleep_split: str

    Time to split the sleep day. Default is "18:00", which is 6:00PM.
num_sleeps: int

    Cutoff for number of sleeps to display in first resulting frame.
    Default = 3, frame will store days with 3+ sleep instances

#### Returns

    short_frame: pd.DataFrame

        Onset, offset, and duration for sleep periods on days with
        more than num_sleeps number of sleep periods
    timing_data: pd.DataFrame

        Modified timing data with included sleep information

#### Example


An example of the short frame and a slice of the added sections to the timing data include:

In [ ]:
short_frame = process_sleep_data(timing_data)[0]
short_frame.head()

Sleep period         Sleep onset  \
UID                   DT                                             
base_v1\user1234      2018-06-28             0 2018-06-29 00:40:30   
                      2018-06-28             2 2018-06-29 13:42:00   
                      2018-06-28             1 2018-06-29 08:50:30   
follow_up_v3\user1234 2018-09-28             1 2018-09-28 22:19:30   
                      2018-09-28             0 2018-09-28 20:07:00   

                                        Sleep offset  Sleep duration  
UID                   DT                                              
base_v1\user1234      2018-06-28 2018-06-29 06:41:00 0 days 06:00:30  
                      2018-06-28 2018-06-29 15:23:30 0 days 01:41:30  
                      2018-06-28 2018-06-29 09:04:00 0 days 00:13:30  
follow_up_v3\user1234 2018-09-28 2018-09-29 06:51:00 0 days 08:31:30  
                      2018-09-28 2018-09-28 21:17:30 0 days 01:10:30

In [ ]:
timing_data = process_sleep_data(timing_data)[1]
timing_data[
    ["Sleep onset", "Sleep offset",
     "Sleep duration", "Sleep onset MSLM",
     "Sleep offset MSLM"]
    ].head()

,Sleep onset,Sleep offset,Sleep duration,Sleep onset MSLM,Sleep offset MSLM
0,2018-07-09 23:38:30,2018-07-10 06:57:00,0 days 07:18:30,1418.5,417.0
1,2018-07-08 00:03:30,2018-07-08 06:35:30,0 days 06:32:00,1443.5,395.5
2,2018-07-10 00:00:00,2018-07-10 00:00:00,0 days 00:00:00,0.0,0.0
4,2018-06-29 22:20:00,2018-06-30 07:44:00,0 days 09:24:00,1340.0,464.0
5,2018-07-07 00:08:00,2018-07-07 06:53:30,0 days 06:45:30,1448.0,413.5


Don't forget to export the data once finished.

In [ ]:
export_timing_data(timing_data)